# Driver Splits

Script to rebase the split times for a stage and display them relative to a specified driver.

In [250]:
import notebookimport

if __name__=='__main__':
    rebase='overallleader'
    rebase='PAD'#'PAD'
    MAXINSPLITDELTA=20 #set xlim on the within split delta
    ss='SS14'
    drivercode=rebase

In [251]:
sr = __import__("Charts - Stage Results")
ssd = __import__("Charts - Split Sector Delta")

In [252]:
import os
import sqlite3
import pandas as pd
import pytablewriter
import six

#dbname='wrc18.db'
#dbname='france18.db'
#conn = sqlite3.connect(dbname)

if __name__=='__main__':
    #dbname='wrc18.db'
    dbname='italy18.db'
    conn = sqlite3.connect(dbname)
    rally='Italy'
    rc='RC1'
    year=2018
    #ss='SS4'

In [253]:
stageresults = sr.dbGetRallyStages(conn, rally).sort_values('number')
stageresults.head()

,legDate,date,startListId,status,itineraryLegId,itinerarySectionId,section,order,code,distance,eventId,name,number,stageId,stageType,status,timingPrecision,itineraryLegId,itinerarySections.itinerarySectionId
0,2018-06-07,Thursday 7th June,153.0,Completed,92,198,Section 1,1,SS1,2.00,32,ITTIRI ARENA SHOW,1,500,HeadToHeadSuperSpecialStage,Completed,Tenth,92,198
1,2018-06-08,Friday 8th June,157.0,Completed,91,199,Section 2,2,SS2,22.12,32,TULA 1,2,502,SpecialStage,Completed,Tenth,91,199
2,2018-06-08,Friday 8th June,157.0,Completed,91,199,Section 2,2,SS3,14.37,32,CASTELSARDO 1,3,503,SpecialStage,Completed,Tenth,91,199
3,2018-06-08,Friday 8th June,157.0,Completed,91,199,Section 2,2,SS4,14.14,32,TERGU - OSILO 1,4,505,SpecialStage,Completed,Tenth,91,199
4,2018-06-08,Friday 8th June,157.0,Completed,91,199,Section 2,2,SS5,11.46,32,MONTE BARANTA 1,5,506,SpecialStage,Completed,Tenth,91,199


In [254]:
stagerank_overall.columns

Index(['diffFirst', 'diffFirstMs', 'diffPrev', 'diffPrevMs', 'entryId',
       'penaltyTime', 'penaltyTimeMs', 'position', 'stageTime', 'stageTimeMs',
       'totalTime', 'totalTimeMs', 'stageId', 'class', 'code', 'distance',
       'name', 'snum', 'driver.code', 'entrant.name', 'classrank',
       'gainedClassPos', 'gainedClassLead', 'classPosDiff', 'lostClassLead',
       'retainedClassLead', 'gainedOverallPos', 'gainedOverallLead',
       'overallPosDiff', 'lostOverallLead', 'retainedOverallLead', 'stagewin',
       'stagewincount', 'winsinarow', 'gainedTime', 'rebased'],
      dtype='object')

In [274]:
def rebaseOverallRallyTime(stagerank_overall, drivercode, code):
    rebase = stagerank_overall[stagerank_overall['driver.code']==drivercode][['code','totalTimeMs']].set_index('code').to_dict(orient='dict')['totalTimeMs']
    stagerank_overall['rebased'] = stagerank_overall['totalTimeMs'] - stagerank_overall['code'].map(rebase)
    return stagerank_overall

In [275]:
zz=rebaseOverallRallyTime(stagerank_overall, 'PAD', ss)
zz=zz[zz['code']==ss][['driver.code','position','totalTimeMs','code', 'rebased']].set_index('driver.code')
zz['Overall Time']=-zz['rebased']/1000
zz

,position,totalTimeMs,code,rebased,Overall Time
driver.code,,,,,
OGI,1,8877600,SS14,-78700,78.7
NEU,2,8884400,SS14,-71900,71.9
LAT,3,8922300,SS14,-34000,34.0
LAP,4,8926800,SS14,-29500,29.5
PAD,5,8956300,SS14,0,0.0
OST,6,8961800,SS14,5500,-5.5
BRE,7,9016800,SS14,60500,-60.5
PRO,9,9488800,SS14,532500,-532.5
TÄN,10,9502300,SS14,546000,-546.0


In [276]:
stagerank_overall = sr.getEnrichedStageRank(conn, rally, typ='overall')
stagerank_overall[stagerank_overall['code']==ss][['driver.code','position','totalTimeMs','code']]

,driver.code,position,totalTimeMs,code
195,OGI,1,8877600,SS14
196,NEU,2,8884400,SS14
197,LAT,3,8922300,SS14
198,LAP,4,8926800,SS14
199,PAD,5,8956300,SS14
200,OST,6,8961800,SS14
201,BRE,7,9016800,SS14
202,PRO,9,9488800,SS14
203,TÄN,10,9502300,SS14
204,SUN,12,9707400,SS14


In [277]:
stagerank_stage = sr.getEnrichedStageRank(conn, rally, typ='stage')
stagerank_stage[stagerank_stage['driver.code']=='PAD'][['position','code']]

,position,code
4,5.0,SS1
24,10.0,SS2
35,6.0,SS3
53,9.0,SS4
68,9.0,SS5
79,5.0,SS6
99,10.0,SS7
112,8.0,SS8
125,6.0,SS9
143,9.0,SS10


In [278]:
splits = ssd.dbGetSplits(conn,rally,ss,rc)

elapseddurations=ssd.getElapsedDurations(splits)
elapseddurations.head()

,drivercode,elapsedDurationS,startDateTime,section
0,EVA,85.5,2018-06-09T14:08:00,1
1,EVA,152.7,2018-06-09T14:08:00,2
2,EVA,229.6,2018-06-09T14:08:00,3
3,EVA,301.7,2018-06-09T14:08:00,4
4,EVA,371.2,2018-06-09T14:08:00,5


In [279]:
roadPos=splits[['drivercode','startDateTime']].drop_duplicates()
roadPos = roadPos.set_index('drivercode')
roadPos['Road Position']=roadPos['startDateTime'].rank().astype(int).astype(str)
roadPos

,startDateTime,Road Position
drivercode,,
EVA,2018-06-09T14:08:00,1
BRE,2018-06-09T14:11:00,2
PAD,2018-06-09T14:14:00,3
OST,2018-06-09T14:17:00,4
LAP,2018-06-09T14:20:00,5
LAT,2018-06-09T14:23:00,6
NEU,2018-06-09T14:26:00,7
OGI,2018-06-09T14:29:00,8
TÄN,2018-06-09T14:32:00,9


In [280]:
splits[splits['drivercode'].isin(['PAD','NEU'])]

,elapsedDuration,elapsedDurationMs,entryId,splitDateTime,splitDateTimeLocal,splitPointId,splitPointTimeId,stageTimeDuration,stageTimeDurationMs,startDateTime,startDateTimeLocal,stageId,class,code,distance,name,drivercode,elapsedDurationS
10,PT1M24.139S,84139,2280,2018-06-09T14:15:24.139Z,2018-06-09T16:15:24.139+02:00,1260,35054,00:07:52.8000000,472800.0,2018-06-09T14:14:00,2018-06-09T16:14:00+02:00,516,RC1,SS14,14.95,COILUNA - LOELLE 2,PAD,84.1
11,PT2M30.226S,150226,2280,2018-06-09T14:16:30.226Z,2018-06-09T16:16:30.226+02:00,1262,35056,00:07:52.8000000,472800.0,2018-06-09T14:14:00,2018-06-09T16:14:00+02:00,516,RC1,SS14,14.95,COILUNA - LOELLE 2,PAD,150.2
12,PT3M46.702S,226702,2280,2018-06-09T14:17:46.702Z,2018-06-09T16:17:46.702+02:00,1265,35058,00:07:52.8000000,472800.0,2018-06-09T14:14:00,2018-06-09T16:14:00+02:00,516,RC1,SS14,14.95,COILUNA - LOELLE 2,PAD,226.7
13,PT4M58.223S,298223,2280,2018-06-09T14:18:58.223Z,2018-06-09T16:18:58.223+02:00,1266,35060,00:07:52.8000000,472800.0,2018-06-09T14:14:00,2018-06-09T16:14:00+02:00,516,RC1,SS14,14.95,COILUNA - LOELLE 2,PAD,298.2
14,PT6M6.9S,366900,2280,2018-06-09T14:20:06.9Z,2018-06-09T16:20:06.9+02:00,1276,35061,00:07:52.8000000,472800.0,2018-06-09T14:14:00,2018-06-09T16:14:00+02:00,516,RC1,SS14,14.95,COILUNA - LOELLE 2,PAD,366.9
30,PT1M22.717S,82717,2279,2018-06-09T14:27:22.717Z,2018-06-09T16:27:22.717+02:00,1260,35073,00:07:46.7000000,466700.0,2018-06-09T14:26:00,2018-06-09T16:26:00+02:00,516,RC1,SS14,14.95,COILUNA - LOELLE 2,NEU,82.7
31,PT2M27.668S,147668,2279,2018-06-09T14:28:27.668Z,2018-06-09T16:28:27.668+02:00,1262,35076,00:07:46.7000000,466700.0,2018-06-09T14:26:00,2018-06-09T16:26:00+02:00,516,RC1,SS14,14.95,COILUNA - LOELLE 2,NEU,147.7
32,PT3M42.253S,222253,2279,2018-06-09T14:29:42.253Z,2018-06-09T16:29:42.253+02:00,1265,35078,00:07:46.7000000,466700.0,2018-06-09T14:26:00,2018-06-09T16:26:00+02:00,516,RC1,SS14,14.95,COILUNA - LOELLE 2,NEU,222.3
33,PT4M52.2S,292200,2279,2018-06-09T14:30:52.2Z,2018-06-09T16:30:52.2+02:00,1266,35080,00:07:46.7000000,466700.0,2018-06-09T14:26:00,2018-06-09T16:26:00+02:00,516,RC1,SS14,14.95,COILUNA - LOELLE 2,NEU,292.2
34,PT6M1.397S,361397,2279,2018-06-09T14:32:01.397Z,2018-06-09T16:32:01.397+02:00,1276,35081,00:07:46.7000000,466700.0,2018-06-09T14:26:00,2018-06-09T16:26:00+02:00,516,RC1,SS14,14.95,COILUNA - LOELLE 2,NEU,361.4


In [281]:
rebasedelapseddurations = ssd.rebaseElapsedDurations(elapseddurations, drivercode)
rebasedelapseddurations.head()

,drivercode,elapsedDurationS,startDateTime,section,rebased
0,EVA,85.5,2018-06-09T14:08:00,1,1.4
1,EVA,152.7,2018-06-09T14:08:00,2,2.5
2,EVA,229.6,2018-06-09T14:08:00,3,2.9
3,EVA,301.7,2018-06-09T14:08:00,4,3.5
4,EVA,371.2,2018-06-09T14:08:00,5,4.3


In [282]:
rbe=-rebasedelapseddurations.pivot('drivercode','section','rebased')
rbe.columns=list(rbe.columns)[:-1]+['{} Overall'.format(ss)]
rbe=rbe.sort_values(rbe.columns[-1],ascending =False)
rbe

,1,2,3,4,5,SS14 Overall
drivercode,,,,,,
OGI,1.7,2.7,4.9,6.9,7.9,10.0
TÄN,1.3,3.1,4.9,6.6,7.9,9.7
MIK,1.6,2.7,4.6,6.1,6.8,8.1
LAP,1.2,2.1,2.7,4.7,6.5,6.6
NEU,1.4,2.5,4.4,6.0,5.5,6.1
LAT,0.7,0.9,2.1,3.1,4.3,5.8
SUN,1.0,1.3,2.1,3.8,3.3,3.7
PAD,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
OST,-0.2,-0.4,-0.8,-0.4,-0.2,-0.6


In [283]:
#https://pandas.pydata.org/pandas-docs/stable/style.html
def color_negative(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    if isinstance(val, str): return ''
    elif val and (isinstance(val,int) or isinstance(val,float)):
        color = 'green' if val < 0 else 'red' if val > 0  else 'black'
    else:
        color='white'
    return 'color: %s' % color

In [284]:
s = rbe.style.applymap(color_negative)
s

,1,2,3,4,5,SS14 Overall
drivercode,,,,,,
OGI,1.7,2.7,4.9,6.9,7.9,10
TÄN,1.3,3.1,4.9,6.6,7.9,9.7
MIK,1.6,2.7,4.6,6.1,6.8,8.1
LAP,1.2,2.1,2.7,4.7,6.5,6.6
NEU,1.4,2.5,4.4,6,5.5,6.1
LAT,0.7,0.9,2.1,3.1,4.3,5.8
SUN,1,1.3,2.1,3.8,3.3,3.7
PAD,-0,-0,-0,-0,-0,-0
OST,-0.2,-0.4,-0.8,-0.4,-0.2,-0.6


In [285]:
splitdurations = ssd.getSplitDurationsFromSplits(conn,rally,ss,rc)
splitdurations.head()

,drivercode,splitDurationS,startDateTime,stageTimeDurationMs,section
0,EVA,85.5,2018-06-09T14:08:00,477700.0,1
1,EVA,67.2,2018-06-09T14:08:00,477700.0,2
2,EVA,76.9,2018-06-09T14:08:00,477700.0,3
3,EVA,72.1,2018-06-09T14:08:00,477700.0,4
4,EVA,69.5,2018-06-09T14:08:00,477700.0,5


In [286]:
rebasedSplits = ssd.rebaseSplitDurations(splitdurations, drivercode)
rebasedSplits.head()

,drivercode,splitDurationS,startDateTime,stageTimeDurationMs,section,rebased
0,EVA,85.5,2018-06-09T14:08:00,477700.0,1,1.4
1,EVA,67.2,2018-06-09T14:08:00,477700.0,2,1.1
2,EVA,76.9,2018-06-09T14:08:00,477700.0,3,0.4
3,EVA,72.1,2018-06-09T14:08:00,477700.0,4,0.6
4,EVA,69.5,2018-06-09T14:08:00,477700.0,5,0.8


In [287]:
splitdurations[splitdurations['drivercode'].isin( ['PAD','NEU'])]

,drivercode,splitDurationS,startDateTime,stageTimeDurationMs,section,rebased
10,PAD,84.1,2018-06-09T14:14:00,472800.0,1,0.0
11,PAD,66.1,2018-06-09T14:14:00,472800.0,2,0.0
12,PAD,76.5,2018-06-09T14:14:00,472800.0,3,0.0
13,PAD,71.5,2018-06-09T14:14:00,472800.0,4,0.0
14,PAD,68.7,2018-06-09T14:14:00,472800.0,5,0.0
71,PAD,105.9,2018-06-09T14:14:00,472800.0,6,0.0
30,NEU,82.7,2018-06-09T14:26:00,466700.0,1,-1.4
31,NEU,65.0,2018-06-09T14:26:00,466700.0,2,-1.1
32,NEU,74.6,2018-06-09T14:26:00,466700.0,3,-1.9
33,NEU,69.9,2018-06-09T14:26:00,466700.0,4,-1.6


In [288]:
rbp=-rebasedSplits.pivot('drivercode','section','rebased')
rbp.columns=['D{}'.format(c) for c in rbp.columns]
rbp.sort_values(rbp.columns[-1],ascending =True)

,D1,D2,D3,D4,D5,D6
drivercode,,,,,,
PRO,-4.8,-6.6,-5.0,-4.1,-5.3,-3.1
BRE,0.1,0.1,-0.2,-0.5,-1.1,-1.7
EVA,-1.4,-1.1,-0.4,-0.6,-0.8,-0.6
OST,-0.2,-0.2,-0.4,0.4,0.2,-0.4
PAD,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
LAP,1.2,0.9,0.6,2.0,1.8,0.1
SUN,1.0,0.3,0.8,1.6,-0.5,0.4
NEU,1.4,1.1,1.9,1.6,-0.5,0.6
MIK,1.6,1.1,1.8,1.5,0.7,1.3


In [289]:
roadPos

,startDateTime,Road Position
drivercode,,
EVA,2018-06-09T14:08:00,1
BRE,2018-06-09T14:11:00,2
PAD,2018-06-09T14:14:00,3
OST,2018-06-09T14:17:00,4
LAP,2018-06-09T14:20:00,5
LAT,2018-06-09T14:23:00,6
NEU,2018-06-09T14:26:00,7
OGI,2018-06-09T14:29:00,8
TÄN,2018-06-09T14:32:00,9


In [290]:
rb2=pd.merge(rbe,rbp,left_index=True, right_index=True)
rb2=pd.merge(rb2,zz[['position','Overall Time']],left_index=True, right_index=True)
rb2=pd.merge(rb2,roadPos[['Road Position']],left_index=True, right_index=True)
cols=rb2.columns.tolist()
rb2=rb2[[cols[-1]]+cols[:-1]]
rb2

,Road Position,1,2,3,4,5,SS14 Overall,D1,D2,D3,D4,D5,D6,position,Overall Time
OGI,8,1.7,2.7,4.9,6.9,7.9,10.0,1.7,1.0,2.3,2.0,1.0,2.1,1,78.7
TÄN,9,1.3,3.1,4.9,6.6,7.9,9.7,1.3,1.8,1.8,1.7,1.3,1.8,10,-546.0
MIK,11,1.6,2.7,4.6,6.1,6.8,8.1,1.6,1.1,1.8,1.5,0.7,1.3,15,-1208.1
LAP,5,1.2,2.1,2.7,4.7,6.5,6.6,1.2,0.9,0.6,2.0,1.8,0.1,4,29.5
NEU,7,1.4,2.5,4.4,6.0,5.5,6.1,1.4,1.1,1.9,1.6,-0.5,0.6,2,71.9
LAT,6,0.7,0.9,2.1,3.1,4.3,5.8,0.7,0.2,1.2,1.0,1.2,1.5,3,34.0
SUN,10,1.0,1.3,2.1,3.8,3.3,3.7,1.0,0.3,0.8,1.6,-0.5,0.4,12,-751.1
PAD,3,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,5,0.0
OST,4,-0.2,-0.4,-0.8,-0.4,-0.2,-0.6,-0.2,-0.2,-0.4,0.4,0.2,-0.4,6,-5.5
BRE,2,0.1,0.2,-0.0,-0.4,-1.5,-3.3,0.1,0.1,-0.2,-0.5,-1.1,-1.7,7,-60.5


In [291]:
#There seems to be missing tenths?
#Elapsed durations are provided in milliseconds. Need to round correctly to tenths?
#Elapsed times grabbed from ssd.dbGetSplits(conn,rally,ss,rc)
from numpy import NaN
rb2=rb2.replace(0,NaN)
rb2=rb2.fillna('')
rb2['position']=rb2['position'].astype(str)
rb2.rename(columns={'position': 'Overall Position'}, inplace=True)
s=rb2.style.applymap(color_negative)
#data.style.applymap(highlight_cols, subset=pd.IndexSlice[:, ['B', 'C']])

s.set_caption("{}: running split times and deltas within each split.".format(ss))


,Road Position,1,2,3,4,5,SS14 Overall,D1,D2,D3,D4,D5,D6,Overall Position,Overall Time
OGI,8,1.7,2.7,4.9,6.9,7.9,10,1.7,1,2.3,2,1,2.1,1,78.7
TÄN,9,1.3,3.1,4.9,6.6,7.9,9.7,1.3,1.8,1.8,1.7,1.3,1.8,10,-546
MIK,11,1.6,2.7,4.6,6.1,6.8,8.1,1.6,1.1,1.8,1.5,0.7,1.3,15,-1208.1
LAP,5,1.2,2.1,2.7,4.7,6.5,6.6,1.2,0.9,0.6,2,1.8,0.1,4,29.5
NEU,7,1.4,2.5,4.4,6,5.5,6.1,1.4,1.1,1.9,1.6,-0.5,0.6,2,71.9
LAT,6,0.7,0.9,2.1,3.1,4.3,5.8,0.7,0.2,1.2,1,1.2,1.5,3,34
SUN,10,1,1.3,2.1,3.8,3.3,3.7,1,0.3,0.8,1.6,-0.5,0.4,12,-751.1
PAD,3,,,,,,,,,,,,,5,
OST,4,-0.2,-0.4,-0.8,-0.4,-0.2,-0.6,-0.2,-0.2,-0.4,0.4,0.2,-0.4,6,-5.5
BRE,2,0.1,0.2,,-0.4,-1.5,-3.3,0.1,0.1,-0.2,-0.5,-1.1,-1.7,7,-60.5


In [292]:
from IPython.display import HTML

styles = [
    dict(selector="caption", props=[("caption-side", "bottom")])
]
html = (df.style.set_table_styles(styles)
          .set_caption("Hover to highlight."))
html

NameError: name 'df' is not defined